<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/predict_show_gradient_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBMで予測

特徴量

|分類	|項目 |
|---|---|
|馬情報	|馬番 |
| |枠番 |
| |年齢 |
| |性別 |
| |体重（現在） | 
| |体重（前走との差分） |
| |負担重量 |
| 当日レース情報 |レース場 |
| |出走馬数 |
| |コース距離 |
| |周回方向 |
| |コースタイプ（ダ/芝/障） |
| |天気 |
| |馬場状態 |
| |開始時間帯 |
| |時期 |
|同馬の過去レース情報（×5走分）	|オッズ |
| |人気 |
| |順位 |
| |タイム（秒） |
| |前走からの経過日数 |
| |コース距離 |
| |コースタイプ（ダ/芝/障） |
| |天気 |
| |馬場状態 |

# 概要

- ライブラリ・データ読み込み
- 前処理
- 学習
- 予測
- 考察

# ライブラリ・データ読み込み

In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
import collections

import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = '/content/drive/My Drive/data/horse-race/'

In [3]:
df = pd.read_csv(DATA_PATH + 'train_gb.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,32,33,34,41,42,43,50,51,52,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,race_id,horse_number,frame_number,age,gen,weight,weight_diff,burden_weight,place,race_horse_number,distance,clockwise,field_type,field_condition,weather,time_hour,season,one_before_odd,one_before_popular,one_before_rank,one_before_time,one_before_elapsed_day,one_before_distance,one_before_field_type,one_before_field_condition,one_before_weather,two_before_odd,two_before_popular,two_before_rank,two_before_time,two_before_elapsed_day,two_before_distance,two_before_field_type,two_before_field_condition,two_before_weather,three_before_odd,three_before_popular,three_before_rank,three_before_time,three_before_elapsed_day,three_before_distance,three_before_field_type,three_before_field_condition,three_before_weather,four_before_odd,four_before_popular,four_before_rank,four_before_time,four_before_elapsed_day,four_before_distance,four_before_field_type,four_before_field_condition,four_before_weather,five_before_odd,five_before_popular,five_before_rank,five_before_time,five_before_elapsed_day,five_before_distance,five_before_field_type,five_before_field_condition,five_before_weather,show,date,rank
0,201945010102,1.0,1.0,7.0,牝,448,0,54.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,2019/1/1,6
1,201945010102,2.0,2.0,7.0,牡,464,7,56.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0,2019/1/1,3
2,201945010102,3.0,3.0,7.0,牝,464,4,54.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0,2019/1/1,1
3,201945010102,4.0,4.0,6.0,牡,449,7,55.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,2019/1/1,8
4,201945010102,5.0,5.0,7.0,牡,502,1,56.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0,2019/1/1,2


# 前処理

## 日付削除

日付はソートのために利用するだけのため削除

In [5]:
df = df.drop('date', axis=1)

## 欠損値

0埋めする。

In [6]:
df.isnull().sum().sum()

9529648

In [7]:
df = df.fillna(0)

過去レースの情報がない馬も結構いるかもしれない。

23752個のデータで過去レースの情報がない。
数値と文字列の0が混在している。

In [8]:
past_race_columns_base = ['odd', 'popular', 'rank', 'time', 'elapsed_day', 'distance', 'field_type', 'field_condition', 'weather']
past_race_num = ['one', 'two', 'three', 'four', 'five']
past_race_columns = []
for n in past_race_num:
  for c in past_race_columns_base:
    past_race_columns.append('{}_before_{}'.format(n, c))

## weight

計測不能が混じっている。

In [9]:
len(df[df['weight'] == '計不'])

44

In [10]:
df[df['weight'] == '計不'] = 0

In [11]:
df['weight'] = df['weight'].astype('int64')

## weight diff

型を変換する

In [12]:
df['weight_diff'] = df['weight_diff'].astype('int64')

## 馬データのない行を削除

In [13]:
df = df[df['horse_number'] != 0.0]

In [14]:
df.head(20)

,race_id,horse_number,frame_number,age,gen,weight,weight_diff,burden_weight,place,race_horse_number,distance,clockwise,field_type,field_condition,weather,time_hour,season,one_before_odd,one_before_popular,one_before_rank,one_before_time,one_before_elapsed_day,one_before_distance,one_before_field_type,one_before_field_condition,one_before_weather,two_before_odd,two_before_popular,two_before_rank,two_before_time,two_before_elapsed_day,two_before_distance,two_before_field_type,two_before_field_condition,two_before_weather,three_before_odd,three_before_popular,three_before_rank,three_before_time,three_before_elapsed_day,three_before_distance,three_before_field_type,three_before_field_condition,three_before_weather,four_before_odd,four_before_popular,four_before_rank,four_before_time,four_before_elapsed_day,four_before_distance,four_before_field_type,four_before_field_condition,four_before_weather,five_before_odd,five_before_popular,five_before_rank,five_before_time,five_before_elapsed_day,five_before_distance,five_before_field_type,five_before_field_condition,five_before_weather,show,rank
0,201945010102,1.0,1.0,7.0,牝,448,0,54.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,6
1,201945010102,2.0,2.0,7.0,牡,464,7,56.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0,3
2,201945010102,3.0,3.0,7.0,牝,464,4,54.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0,1
3,201945010102,4.0,4.0,6.0,牡,449,7,55.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,8
4,201945010102,5.0,5.0,7.0,牡,502,1,56.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0,2
5,201945010102,6.0,6.0,6.0,牡,445,-1,56.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,9
6,201945010102,7.0,7.0,4.0,牡,481,-14,56.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,7
7,201945010102,8.0,7.0,7.0,牝,399,3,54.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,4
8,201945010102,9.0,8.0,8.0,牝,392,-2,51.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,10
9,201945010102,10.0,8.0,4.0,牝,452,32,54.0,川崎,10.0,1400.0,左,ダ,良,晴,11.0,winter,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,5


## 過去レースの情報が全くない馬を削除

In [14]:
len(df[df['one_before_odd']==0.0])

23748

In [18]:
df = df[df['one_before_odd'] != 0.0]

## Label Encoding

In [19]:
categorical_cols = ['gen', 'place', 'clockwise', 'field_type', 'field_condition', 'weather', 'season']
categorical_cols_past_base = ['field_type', 'field_condition', 'weather']

In [20]:
categorical_cols_past = []
for n in past_race_num:
  for c in categorical_cols_past_base:
    categorical_cols_past.append('{}_before_{}'.format(n, c))

In [21]:
categorical_cols = categorical_cols + categorical_cols_past

In [22]:
df[categorical_cols] = df[categorical_cols].astype(str)

In [23]:
for c in categorical_cols:
  le = LabelEncoder()
  le.fit(df[c])
  df[c] = le.transform(df[c])

# 学習

## 学習データと検証データに分割

過去レースの馬を削除しない場合は53584

削除した場合は49840

In [33]:
train_df, test_df = train_test_split(df, test_size=49840, shuffle=False)

In [34]:
test_df

,race_id,horse_number,frame_number,age,gen,weight,weight_diff,burden_weight,place,race_horse_number,distance,clockwise,field_type,field_condition,weather,time_hour,season,one_before_odd,one_before_popular,one_before_rank,one_before_time,one_before_elapsed_day,one_before_distance,one_before_field_type,one_before_field_condition,one_before_weather,two_before_odd,two_before_popular,two_before_rank,two_before_time,two_before_elapsed_day,two_before_distance,two_before_field_type,two_before_field_condition,two_before_weather,three_before_odd,three_before_popular,three_before_rank,three_before_time,three_before_elapsed_day,three_before_distance,three_before_field_type,three_before_field_condition,three_before_weather,four_before_odd,four_before_popular,four_before_rank,four_before_time,four_before_elapsed_day,four_before_distance,four_before_field_type,four_before_field_condition,four_before_weather,five_before_odd,five_before_popular,five_before_rank,five_before_time,five_before_elapsed_day,five_before_distance,five_before_field_type,five_before_field_condition,five_before_weather,show,rank
325314,202047031706,1.0,1.0,3.0,1,454,7,54.0,18,9.0,1400.0,0,0,2,2,13.0,1,1.0,1.0,2.0,989.0,28.0,1400.0,0,3,4,296.1,14.0,11.0,175.0,72.0,1200.0,2,4,4,82.6,10.0,8.0,474.0,170.0,1200.0,2,4,5,24.7,9.0,10.0,773.0,100.0,1200.0,2,3,4,230.8,12.0,9.0,96.0,128.0,1600.0,3,4,4,0.0,7
325315,202047031706,2.0,2.0,3.0,1,386,1,54.0,18,9.0,1400.0,0,0,2,2,13.0,1,10.6,6.0,2.0,988.0,41.0,1400.0,0,2,2,4.0,2.0,5.0,889.0,26.0,1400.0,2,4,5,10.0,4.0,5.0,189.0,54.0,1400.0,2,2,6,2.2,1.0,2.0,661.0,173.0,1000.0,2,5,4,20.6,5.0,3.0,562.0,216.0,1000.0,2,3,5,1.0,2
325316,202047031706,3.0,3.0,3.0,1,437,7,54.0,18,9.0,1400.0,0,0,2,2,13.0,1,5.2,4.0,3.0,689.0,26.0,1400.0,0,2,3,14.6,6.0,2.0,690.0,56.0,1400.0,2,4,4,20.5,5.0,4.0,292.0,67.0,1400.0,2,4,4,2.0,1.0,2.0,961.0,175.0,1000.0,2,5,4,7.0,2.0,2.0,662.0,189.0,1000.0,2,4,4,0.0,6
325317,202047031706,4.0,4.0,3.0,0,460,5,56.0,18,9.0,1400.0,0,0,2,2,13.0,1,6.2,3.0,5.0,889.0,41.0,1400.0,0,2,2,4.7,3.0,2.0,689.0,26.0,1400.0,2,4,5,4.4,2.0,3.0,988.0,54.0,1400.0,2,2,6,41.5,9.0,12.0,872.0,262.0,1150.0,2,3,5,6.5,4.0,1.0,890.0,102.0,1400.0,2,4,4,0.0,5
325318,202047031706,5.0,5.0,3.0,2,503,6,56.0,18,9.0,1400.0,0,0,2,2,13.0,1,19.4,7.0,1.0,290.0,28.0,1400.0,0,3,3,14.1,3.0,4.0,393.0,33.0,1400.0,2,5,5,7.5,3.0,5.0,292.0,67.0,1400.0,2,4,4,45.6,8.0,9.0,519.0,183.0,1800.0,2,4,4,56.4,9.0,9.0,412.0,220.0,1700.0,2,4,4,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407327,202048060911,6.0,6.0,5.0,1,419,1,54.0,5,10.0,1600.0,0,0,2,2,17.0,2,153.0,8.0,6.0,791.0,34.0,1400.0,0,2,3,22.6,5.0,1.0,690.0,12.0,1400.0,2,4,4,98.5,8.0,8.0,492.0,25.0,1400.0,2,4,4,112.2,7.0,7.0,692.0,46.0,1400.0,2,4,4,49.6,8.0,6.0,492.0,60.0,1400.0,2,4,4,0.0,5
407328,202048060911,7.0,7.0,4.0,0,486,6,56.0,5,10.0,1600.0,0,0,2,2,17.0,2,1.6,1.0,1.0,390.0,34.0,1400.0,0,2,3,1.3,1.0,1.0,688.0,26.0,1400.0,2,4,4,8.8,4.0,2.0,590.0,280.0,1400.0,2,4,4,3.8,3.0,1.0,988.0,266.0,1400.0,2,4,4,4.2,2.0,2.0,191.0,293.0,1400.0,2,4,4,1.0,2
407329,202048060911,8.0,7.0,5.0,1,424,-5,50.0,5,10.0,1600.0,0,0,2,2,17.0,2,2.4,1.0,8.0,406.0,35.0,1600.0,0,2,2,4.9,3.0,1.0,804.0,12.0,1600.0,2,4,4,5.2,4.0,5.0,805.0,26.0,1600.0,2,4,4,1.6,1.0,1.0,803.0,307.0,1600.0,2,4,4,2.1,1.0,1.0,305.0,294.0,1600.0,2,4,5,0.0,10
407330,202048060911,9.0,8.0,4.0,1,403,-2,55.0,5,10.0,1600.0,0,0,2,2,17.0,2,3.4,2.0,1.0,188.0,34.0,1400.0,0,2,3,5.8,3.0,1.0,417.0,14.0,1800.0,2,4,5,2.8,2.0,3.0,590.0,46.0,1400.0,2,4,4,1.8,1.0,3.0,91.0,60.0,1400.0,2,4,4,2.5,1.0,4.0,890.0,88.0,1400.0,2,3,5,0.0,9


In [35]:
X_train = train_df.drop(['race_id', 'show', 'rank'], axis=1).values
y_train = train_df['rank'].values.reshape(-1)
X_test = test_df.drop(['race_id', 'show', 'rank'], axis=1).values
y_test = test_df['rank'].values.reshape(-1)

In [36]:
train_race_id_counter = collections.Counter(list(train_df['race_id'].values))
test_race_id_counter = collections.Counter(list(test_df['race_id'].values))
train_query = list(train_race_id_counter.values())
test_query = list(test_race_id_counter.values())

In [37]:
print('train X', X_train.shape)
print('train y', y_train.shape)
print('test X', X_test.shape)
print('test y', y_test.shape)

train X (180004, 61)
train y (180004,)
test X (49840, 61)
test y (49840,)


## 学習

In [43]:
lgb_train = lgb.Dataset(X_train, y_train, group=train_query)
lgb_eval = lgb.Dataset(X_test, y_test, group=test_query)

params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [1, 3],
    'boosting_type': 'gbdt',
}

num_round = 1000

In [44]:
model = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_eval,
    num_boost_round=num_round,
)

[1]	valid_0's ndcg@1: 0.375515	valid_0's ndcg@3: 0.477368
[2]	valid_0's ndcg@1: 0.388954	valid_0's ndcg@3: 0.488512
[3]	valid_0's ndcg@1: 0.393294	valid_0's ndcg@3: 0.492269
[4]	valid_0's ndcg@1: 0.395222	valid_0's ndcg@3: 0.495305
[5]	valid_0's ndcg@1: 0.396148	valid_0's ndcg@3: 0.496187
[6]	valid_0's ndcg@1: 0.40083	valid_0's ndcg@3: 0.498225
[7]	valid_0's ndcg@1: 0.405825	valid_0's ndcg@3: 0.500961
[8]	valid_0's ndcg@1: 0.404018	valid_0's ndcg@3: 0.501182
[9]	valid_0's ndcg@1: 0.404673	valid_0's ndcg@3: 0.502138
[10]	valid_0's ndcg@1: 0.407524	valid_0's ndcg@3: 0.503758
[11]	valid_0's ndcg@1: 0.409501	valid_0's ndcg@3: 0.503536
[12]	valid_0's ndcg@1: 0.410381	valid_0's ndcg@3: 0.504694
[13]	valid_0's ndcg@1: 0.413685	valid_0's ndcg@3: 0.505636
[14]	valid_0's ndcg@1: 0.413595	valid_0's ndcg@3: 0.505737
[15]	valid_0's ndcg@1: 0.412654	valid_0's ndcg@3: 0.505379
[16]	valid_0's ndcg@1: 0.412322	valid_0's ndcg@3: 0.505555
[17]	valid_0's ndcg@1: 0.413259	valid_0's ndcg@3: 0.505931
[18]	va

# 予測

In [45]:
pred = model.predict(X_test)

レースごとに予測された確率が最も高い馬が３位以内に入っている確率を算出する。

In [46]:
def calc_prob(predict):
  stack_q = 0
  correct = 0
  for query in test_query:
    ind = np.argmax(predict[stack_q:stack_q+query])
    stack_q += query
    if test_df.iloc[ind]['show'] == 1:
      correct += 1

  print('score is', correct / len(test_query))

In [47]:
calc_prob(pred)

score is 0.3347382431233363


# 考察